# Orphan Wells in Alberta



Data from [orphanwell.ca](https://www.orphanwell.ca/about/orphan-inventory) as of 2021-12-14.

The Orphan Well Association or OWA makes no representation, warranties, or guarantees, expressed or implied, for the fitness of the data with respect to its use. The OWA is not responsible for any costs incurred from use of the information in this list.

The AER Licence Details Report includes multiple events for individual wells in order to report production. Therefore there will be more entries than number of wells. Users of this list are advised that it does not necessarily reflect the current inventory of orphan wells held by the OWA and users are responsible for confirming any information used.

https://www.wellwiki.org/wiki/Special:AlbertaAdvancedWellSearch

In [2]:
import pandas as pd
try:
    df = pd.read_csv('./orphan-wells-alberta.csv')
except:
    df = pd.read_csv('') # GitHub link here
df

,Unique Well Identifier,Status,Surface Latitude,Surface Longitude
0,100/01-01-021-27W4/0,Closed,50.747744,-113.610782
1,100/16-36-020-27W4/0,Closed,50.747742,-113.611065
2,100/13-31-001-18W4/0,Active,49.087051,-112.410697
3,100/01-01-012-13W4/0,Active,49.963746,-111.647497
4,100/16-36-016-25W4/0,Closed,50.397926,-113.304177
...,...,...,...,...
8678,100/16-36-060-15W4/0,Closed,54.235706,-112.105920
8679,100/16-36-065-23W4/0,Active,54.673830,-113.337140
8680,100/16-36-065-27W4/3,Closed,54.673794,-113.938961
8681,100/16-36-066-24W4/0,Closed,54.762023,-113.486038


There are 8683 rows in this data set, which means that 8683 orphan wells have been identified in Alberta.

We can also check how many wells there are in the data set with each "Status".

In [12]:
for status in df['Status'].unique():
    print(status, df[df['Status'] == status].shape[0])

Closed 6639
Active 2044


In [ ]:
import folium
from folium.plugins import MarkerCluster
m = folium.Map(location=[53.5,-114], zoom_start=5)
mc = MarkerCluster()
for i, row in df.iterrows():
    if row['Status'] == 'Closed':
        color = 'red'
    else:
        color = 'blue'
    mc.add_child(folium.Marker(location=[row['Surface Latitude'],row['Surface Longitude']],tooltip=row['Unique Well Identifier']+' '+row['Status'],icon=folium.Icon(color=color)))
m.add_child(mc)
m

In [1]:
spreadsheet_key = '1iM7SC-UqAHPpPnXCT3X8hxVuZV0KD7CpaUGr13JbVoc'
spreadsheet_gid = '77886383'

import pandas as pd
csv_link = 'https://docs.google.com/spreadsheets/d/'+spreadsheet_key+'/export?gid='+spreadsheet_gid+'&format=csv'
print(csv_link)

https://docs.google.com/spreadsheets/d/1iM7SC-UqAHPpPnXCT3X8hxVuZV0KD7CpaUGr13JbVoc/export?gid=77886383&format=csv


In [ ]:
# without marker clusters, this is terrible

spreadsheet_key = '1iM7SC-UqAHPpPnXCT3X8hxVuZV0KD7CpaUGr13JbVoc'
spreadsheet_gid = '77886383'

import pandas as pd
csv_link = 'https://docs.google.com/spreadsheets/d/'+spreadsheet_key+'/export?gid='+spreadsheet_gid+'&format=csv'
df = pd.read_csv(csv_link)

import folium
from folium.plugins import MarkerCluster
m = folium.Map(location=[53.5,-114], zoom_start=5)
mc = MarkerCluster()
for i, row in df.iterrows():
    if row['Status'] == 'Closed':
        color = 'red'
    else:
        color = 'blue'
    folium.Marker(location=[row['Surface Latitude'],row['Surface Longitude']],tooltip=row['Unique Well Identifier']+' '+row['Status'],icon=folium.Icon(color=color)).add_to(m)
m

In [ ]:
url = 'https://www.wellwiki.org/wiki/Special:AlbertaAdvancedWellSearch?operatorfiltertype=matches&operatorfiltervalue=&countyfiltertype=matches&countyfiltervalue=&datefiltertype=matches&datefiltervalue=&licensefiltertype=matches&licensefiltervalue=Abandoned&meridianfiltertype=matches&meridianfiltervalue=&townshipfiltertype=matches&townshipfiltervalue=&rangefiltertype=matches&rangefiltervalue=&sectionfiltertype=matches&sectionfiltervalue=&pagenum=&pagesize=100000'

import pandas as pd
df = pd.read_html(url)[1]
df.columns = ['UWI','Operator','County/Municipality','Meridian','Township','Range','Section','License Date','License Status']
df['Latitude'] = ''
df['Longitude'] = ''
df

In [ ]:
df.to_csv('abandoned-wells.csv')

In [ ]:
%%time
for i, row in df.head(100).iterrows():
    well_id = row[0]
    loc = pd.read_html('https://www.wellwiki.org/wiki/'+well_id)[0][0]
    lat = loc[19].split(': ')[1]
    lon = loc[20].split(': ')[1]
    df.loc[[i],['Latitude']] = lat
    df.loc[[i],['Longitude']] = lon
df.head(10)

In [ ]:
import folium
from folium.plugins import MarkerCluster
m = folium.Map(location=[53.5,-114], zoom_start=5)
mc = MarkerCluster()
for row in df.head(100).itertuples():
    mc.add_child(folium.Marker(location=[row.Latitude,row.Longitude], tooltip=row.UWI))
m.add_child(mc)
m

In [ ]:
UWI = '0104181914002'
TWP = UWI[:3]
Meridian = UWI[3:4]
Range = UWI[4:6]
Section = UWI[6:8]
LSD = UWI[8:10] # not sure about this one
print('Meridian',Meridian)
print('TWP',TWP)
print('Range',Range)
print('Section',Section)
print('LSD',LSD)